In [1]:
!pip install onnxruntime

import os
import json
import zipfile
import numpy as np
import tensorflow as tf
from keras import utils
import onnxruntime as ort
from datetime import datetime
from sklearn.metrics import accuracy_score, classification_report

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 198.2 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 228.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 176.2 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 267.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.2/536.2 kB 339.6 MB/s eta 0:00:00

[notice] A new release of pip available: 22.2.2 -> 24.2
[notice] To update, run: pip install --upgrade pip


2024-08-28 13:11:01.657664: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-28 13:11:01.657726: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-28 13:11:01.659102: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-28 13:11:01.666827: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-28 13:11:02.577425: W tensorflow/comp

In [11]:
def unzip_file(zip_file_path, extract_to_dir):
    if not os.path.exists(extract_to_dir):
        os.makedirs(extract_to_dir)

    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to_dir)

    print(f"Files extracted to: {extract_to_dir}")

In [12]:
zip_file_path = 'images_zipped.zip'
extract_to_dir = 'train'
unzip_file(zip_file_path, extract_to_dir)

Files extracted to: train


In [14]:
zip_file_path = 'modals.zip'
extract_to_dir = 'modals'
unzip_file(zip_file_path, extract_to_dir)

Files extracted to: modals


In [15]:
path_to_data = 'train/'

In [16]:
val_data=utils.image_dataset_from_directory(
    path_to_data,
    labels="inferred",
    label_mode="int",
    validation_split=0.1,
    subset="validation",
    color_mode="rgb",
    image_size=(256,256),
    batch_size=64,
    seed=40,
)

Found 1129 files belonging to 10 classes.
Using 112 files for validation.


In [17]:
for images,lables in val_data.take(1):
  print('Amount of train images: ', images.shape)
  print('Amount of train labels: ', lables.shape)

Amount of train images:  (64, 256, 256, 3)
Amount of train labels:  (64,)


In [18]:
def normalize(image, label):
  return image/255.0, label

In [19]:
val_data= val_data.map(normalize)

In [20]:
val_x=[]
val_y=[]

for image,label in val_data:
   val_x.append(image)
   val_y.append(label)

val_x = tf.concat(val_x, axis=0)
val_y = tf.concat(val_y, axis=0)

In [21]:
num_classes = 10
val_y = tf.keras.utils.to_categorical(val_y, num_classes=num_classes)

In [22]:
def find_model_file(folder_path):
    for file in os.listdir(folder_path):
        if file.endswith('.onnx'):
            return file

    return None

In [28]:
new_model_name = find_model_file('modals/newmodal')
production_model_name = find_model_file('modals/production')

In [30]:
new_model = f"modals/newmodal/{new_model_name}"
new_model_session = ort.InferenceSession(new_model)

production_model = f"modals/production/{production_model_name}"
production_model_session = ort.InferenceSession(production_model)

new_model_input = new_model_session.get_inputs()[0].name
new_model_output = new_model_session.get_outputs()[0].name

production_model_input = production_model_session.get_inputs()[0].name
production_model_output = production_model_session.get_outputs()[0].name

In [31]:
new_pred_arr = []
production_pred_arr = []

for i, val in enumerate(val_x):
    new_model_pred = new_model_session.run([new_model_output], {new_model_input: [val_x[i].cpu().numpy()]})
    production_model_pred = production_model_session.run([production_model_output], {production_model_input: [val_x[i].cpu().numpy()]})

    new_pred_arr.append(new_model_pred)
    production_pred_arr.append(production_model_pred)

Instructions for updating:
Use tf.identity with explicit device placement instead.


In [32]:
new_pred_arr = np.concatenate([np.argmax(pred[0], axis=1) for pred in new_pred_arr], axis=0)
production_pred_arr = np.concatenate([np.argmax(pred[0], axis=1) for pred in production_pred_arr], axis=0)

val_y_flat = np.argmax(val_y, axis=1)

new_model_accuracy = accuracy_score(val_y_flat, new_pred_arr)
production_model_accuracy = accuracy_score(val_y_flat, production_pred_arr)

New Model Accuracy: 78.57%
Production Model Accuracy: 62.50%

New Model Classification Report:
              precision    recall  f1-score   support

           0       0.89      0.57      0.70        14
           1       0.75      1.00      0.86        21
           2       0.60      0.90      0.72        10
           3       0.80      0.80      0.80        10
           4       1.00      0.78      0.88         9
           5       0.86      0.75      0.80         8
           6       0.40      0.50      0.44         4
           7       0.83      0.50      0.62        10
           8       0.84      0.84      0.84        19
           9       1.00      0.86      0.92         7

    accuracy                           0.79       112
   macro avg       0.80      0.75      0.76       112
weighted avg       0.81      0.79      0.78       112


Production Model Classification Report:
              precision    recall  f1-score   support

           0       0.64      0.50      0.56       

In [34]:
should_replace = new_model_accuracy > production_model_accuracy

with open('should_replace.json', 'w') as f:
    json.dump({'replace': should_replace}, f)

In [36]:
new_model_folder = 'modals/newmodal'
replacement_zip = 'replacement.zip'

with zipfile.ZipFile(replacement_zip, 'w') as zipf:
    zipf.write('should_replace.json')

    if should_replace:
        for root, dirs, files in os.walk(new_model_folder):
            for file in files:
                file_path = os.path.join(root, file)
                arcname = os.path.relpath(file_path, new_model_folder)
                zipf.write(file_path, arcname)

print(f"Files zipped into '{replacement_zip}'.")


Files zipped into 'replacement.zip'.


In [44]:
filename = 'model_comparison.txt'

with open(filename, 'w') as file:
    file.write(f"Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n")
    file.write(f"New Model Name: {new_model_name}\n")
    file.write(f"New Model Accuracy: {new_model_accuracy * 100:.2f}%\n\n")
    file.write(f"Production Model Name: {production_model_name}\n")
    file.write(f"Production Model Accuracy: {production_model_accuracy * 100:.2f}%\n")

    file.write("\nNew Model Classification Report:\n")
    file.write(classification_report(val_y_flat, new_pred_arr, zero_division=0))

    file.write("\nProduction Model Classification Report:\n")
    file.write(classification_report(val_y_flat, production_pred_arr, zero_division=0))

print(f"Results saved to {filename}")

Results saved to model_comparison_20240828_141815.txt
